### Summarization example with Open AI and Langchain

#### This notebook shows how to use Stuff Documents Chain and Map Reduce Chain to summarize web pages url using Open AI API

In [21]:
import warnings
warnings.filterwarnings('ignore')
from bs4 import BeautifulSoup

from configparser import ConfigParser
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.document_loaders import WebBaseLoader

# map reduce libraries:
from langchain.text_splitter import TokenTextSplitter
from langchain.chains import MapReduceDocumentsChain
from langchain.chains import ReduceDocumentsChain

In [4]:
# Get the OpenAI API key in a config object:
config_object = ConfigParser()
config_object.read("../../config_genai.ini")

openai.api_key = config_object["OPENAI"]["openai_api_key"]

In [13]:
# Create LLM
llm = ChatOpenAI(openai_api_key=openai.api_key)

In [ ]:
# Create the prompt template

prompt_template = """Write a concise summary of the following content:

{content}

Summary:
"""
prompt = PromptTemplate.from_template(prompt_template)

In [14]:
# Create the LLM Chain
llm_chain = LLMChain(prompt=prompt, llm=llm)

#### Stuff Documents Chain

* We just put all the content of the document into a single prompt and send that prompt to the GPT API. 

In [15]:

# Call the LLM Chain
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain, 
    document_variable_name="content"
  )


In [ ]:
url_link = 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/'

In [9]:

# load the url through Lanchain WebBaseLoader:
loader = WebBaseLoader(url_link)
docs = loader.load()

In [11]:
# Print to see the docs
docs

[Document(page_content='\n\n\n\n\n\nPrompt Engineering | Lil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLil\'Log\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nPosts\n\n\n\n\nArchive\n\n\n\n\nSearch\n\n\n\n\nTags\n\n\n\n\nFAQ\n\n\n\n\nemojisearch.app\n\n\n\n\n\n\n\n\n\n      Prompt Engineering\n    \nDate: March 15, 2023  |  Estimated Reading Time: 21 min  |  Author: Lilian Weng\n\n\n \n\n\nTable of Contents\n\n\n\nBasic Prompting\n\nZero-Shot\n\nFew-shot\n\nTips for Example Selection\n\nTips for Example Ordering\n\n\n\nInstruction Prompting\n\nSelf-Consistency Sampling\n\nChain-of-Thought (CoT)\n\nTypes of CoT prompts\n\nTips and Extensions\n\n\nAutomatic Prompt Design\n\nAugmented Language Models\n\nRetrieval\n\nProgramming Language\n\nExternal APIs\n\n\nCitation\n\nUseful Resources\n\nReferences\n\n\n\n\n\nPrompt Engineering, also known as In-Context Prompting, refers to methods for how to communicate with LLM to steer its behavior for desired 

In [12]:
# Putting everything in a function:

def summarize_web(prompt_template, url):
    
    # Create LLM
    llm = ChatOpenAI(openai_api_key=openai.api_key)
    prompt = PromptTemplate.from_template(prompt_template)
    llm_chain = LLMChain(prompt=prompt, llm=llm)
    
    
    # Call the LLM Chain
    stuff_chain = StuffDocumentsChain(
        llm_chain=llm_chain, 
        document_variable_name="content"
    )
    
    # load the url through Lanchain WebBaseLoader:
    loader = WebBaseLoader(url)
    docs = loader.load()
    summary = stuff_chain.run(docs)
    return print(summary)

In [2]:
url = 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/'

# Create the prompt template

prompt_template = """Write a concise summary of the following content:

{content}

Summary:
"""

In [6]:
# call the function:
summarize_web(prompt_template, url)

# this url link gives error : This model's maximum context length is 4097 tokens. However, your messages resulted in 7106 tokens. Please reduce the length of the messages

c:\Users\Dipjyoti\.conda\envs\langchain-webapp\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
c:\Users\Dipjyoti\.conda\envs\langchain-webapp\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


BadRequestError: Error code: 400 - {'error': {'message': "This model's maximum context length is 4097 tokens. However, your messages resulted in 7106 tokens. Please reduce the length of the messages.", 'type': 'invalid_request_error', 'param': 'messages', 'code': 'context_length_exceeded'}}

In [10]:
# Use another url which has shorter token length:
url2 = 'https://python.langchain.com/docs/get_started/introduction'

prompt_template2 = """Write a concise summary of the following content 
provide the output in bullent points:

{content}

Summary:
"""

In [13]:
# call the function:
summarize_web(prompt_template2, url2)

- LangChain is a framework for developing applications powered by language models
- It enables context-aware applications that can reason based on provided context
- LangChain consists of several parts including libraries, templates, LangServe, and LangSmith
- The LangChain libraries provide components and off-the-shelf chains for working with language models
- LangChain Expression Language (LCEL) is a declarative way to compose chains in LangChain
- LangChain provides standard interfaces and integrations for modules such as model I/O, retrieval, and agents
- LangChain offers use cases, integrations, guides, API reference, and a developer's guide
- LangChain simplifies the application lifecycle by facilitating development, productionization, and deployment
- LangChain is part of a rich ecosystem of tools and has a growing list of integrations.


#### Map Reduce Chain

* we break the document into smaller pieces and use the map-reduce chain for summarization. 
* First, we summarize each chunk individually, which is the "map" step. Then, we take these individual summaries and combine them into one final summary, known as the "reduce" step.

In the "Reduce Documents Chain," If we try to combine all the summaries from the chunks in one go, we might still run into the token limit issue. This is because individual summaries are usually smaller than the token limit, but when combined, they can exceed it. So, in this chain, we ensure that the summaries of the chunks we combine remain within the token limit. We do it step by step, iteratively reducing the chunks until we get our final summary. It's important to note that we're using the StuffDocumentsChain

In [15]:
# creating the map chain which is a normal LLMChain that combines the PromptTemplate component and the LLM Component.
# first step:
map_template = """Write a concise summary of the following content:

{content}

Summary:
"""
map_prompt = PromptTemplate.from_template(map_template)
# Create LLM
llm = ChatOpenAI(openai_api_key=openai.api_key)
map_chain = LLMChain(prompt=map_prompt, llm=llm)


In [16]:
# second step: we are using StuffDocumentsChain inside the ReduceDocumentsChain
# we imported the StuffDocumentsChain, created a template for the reduce chain which is used during the reduce step. We created a reduce chain which is also an LLMChain and passed it to the StuffDocumentsChain
# We provided the name of the placeholder in the template through the document_variable_name which helps the StuffDocumentsChain to identify where to insert the document summaries.
from langchain.chains.combine_documents.stuff import StuffDocumentsChain

reduce_template = """The following is set of summaries:

{doc_summaries}

Summarize the above summaries with all the key details
Summary:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(prompt=reduce_prompt, llm=llm)
stuff_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="doc_summaries")


In [17]:
# using StuffDocumentsChain to the ReduceDocumentsChain

from langchain.chains import ReduceDocumentsChain

reduce_chain = ReduceDocumentsChain(
    combine_documents_chain=stuff_chain
)

In [18]:
# option to set a maximum token length (default = 3000), which determines how many summaries to include in the reduce step. It's crucial to ensure that this value remains below the token limit provided by the LLM provider.
reduce_chain = ReduceDocumentsChain(
    combine_documents_chain=stuff_chain,
    token_max=3000
)

In [19]:
# With both our map_chain and reduce_chain in place, putting together in map_reduce_chain
from langchain.chains import MapReduceDocumentsChain

map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    document_variable_name="content",
    reduce_documents_chain=reduce_chain
)


In [ ]:
# Load the document,  break it into smaller, token-compliant chunks, and then feed these chunks into map_reduce_chain
# It ensures that each of these chunks remains under the 2000-token limit, used tiktoken module
from langchain.text_splitter import TokenTextSplitter

splitter = TokenTextSplitter(chunk_size=2000)
split_docs = splitter.split_documents(docs)

In [ ]:
# Let's provide those split_docs to the map_reduce_chain 
summary = map_reduce_chain.run(split_docs)
print(summary)

In [23]:
# Putting everything in a function:

def summarize_web_map_reduce(map_template, reduce_template, url):
    
    # Create LLM
    llm = ChatOpenAI(openai_api_key=openai.api_key)
    # Map Chain
    map_prompt = PromptTemplate.from_template(map_template)
    map_chain = LLMChain(prompt=map_prompt, llm=llm)
    # Reduce Chain
    reduce_prompt = PromptTemplate.from_template(reduce_template)
    reduce_chain = LLMChain(prompt=reduce_prompt, llm=llm)
    
    
    
    # Call the LLM Stuff Documents Chain
    stuff_chain = StuffDocumentsChain(llm_chain=reduce_chain, document_variable_name="doc_summaries")
    # reduce chain:
    reduce_chain = ReduceDocumentsChain(combine_documents_chain=stuff_chain,)
    
    # Map Reduce Chain
    map_reduce_chain = MapReduceDocumentsChain(
        llm_chain=map_chain,
        document_variable_name="content",
        reduce_documents_chain=reduce_chain)
    
    # load the content from web url through Langchain WebBaseLoader:
    loader = WebBaseLoader(url)
    docs = loader.load()
    
    # Split the content into smaller chuncks
    splitter = TokenTextSplitter(chunk_size=2000)
    split_docs = splitter.split_documents(docs)
    
    # Use Map reduce chain to summarize
    summary = map_reduce_chain.run(split_docs)
 
    return print(summary)

In [27]:
url1 = 'https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/'
# Create the map template:
map_template = """Write a concise summary of the following content:

{content}

Summary:
"""

# Reduce Chain template
reduce_template = """The following is set of summaries:

{doc_summaries}

Write a concise summary of the the above summaries with all the key details
provide the output in bullet points:
Summary:"""

In [28]:
# Call the summarize web map reduce function:
summarize_web_map_reduce(map_template=map_template, reduce_template=reduce_template, url=url1)
# took around 30 seconds

- The post discusses prompt engineering for autoregressive language models, including zero-shot and few-shot learning, example selection and ordering, and instruction prompting.
- The content emphasizes the importance of specificity and precision in task requirements and provides examples of prompt definitions for different tasks.
- Various prompting techniques are mentioned, such as self-consistency sampling and chain-of-thought prompting, as well as automatic prompt design methods like AutoPrompt and Prompt-Tuning.
- The use of external knowledge, programming language statements, and external APIs in language models is explored.
- Toolformer is a language model that can utilize external tools through simple APIs, trained in a self-supervised manner with a few API demonstrations.
- A list of research papers covering topics related to language models and their applications is provided, highlighting various techniques and approaches for improving their performance and capabilities.


### End